In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# --- 0. path to data files ---
files = {
    "Z1": "Z1_CAJICA_ambient-weather-20250604-20251104.csv",
    "Z2": "Z2_GIRALDA_ambient-weather-20250604-20251104.csv",
    "Z3": "Z3_OIKOS_ambient-weather-20250604-20251104.csv"
}

In [3]:
# --- 1. load and inspect data ---
def load_sample(path, nrows=0):
    if nrows>0:
        return pd.read_csv(path, nrows=nrows)
    return pd.read_csv(path)

# quick inspect
for k,p in files.items():
    df = load_sample(p, nrows=5)
    print(k, df.columns.tolist())

Z1 ['Date', 'Simple Date', 'Outdoor Temperature (°C)', 'Feels Like (°C)', 'Wind Speed (m/sec)', 'Wind Gust (m/sec)', 'Max Daily Gust (m/sec)', 'Wind Direction (°)', 'Rain Rate (mm/hr)', 'Event Rain (mm)', 'Daily Rain (mm)', 'Weekly Rain (mm)', 'Monthly Rain (mm)', 'Yearly Rain (mm)', 'Total Rain (mm)', 'Relative Pressure (mmHg)', 'Humidity (%)', 'Ultra-Violet Radiation Index', 'Solar Radiation (W/m^2)', 'Absolute Pressure (mmHg)']
Z2 ['Date', 'Simple Date', 'Outdoor Temperature (°C)', 'Feels Like (°C)', 'Wind Speed (m/sec)', 'Wind Gust (m/sec)', 'Max Daily Gust (m/sec)', 'Wind Direction (°)', 'Rain Rate (mm/hr)', 'Event Rain (mm)', 'Daily Rain (mm)', 'Weekly Rain (mm)', 'Monthly Rain (mm)', 'Total Rain (mm)', 'Relative Pressure (mmHg)', 'Humidity (%)', 'Ultra-Violet Radiation Index', 'Solar Radiation (W/m^2)', 'Absolute Pressure (mmHg)', 'CO2 battery']
Z3 ['Date', 'Simple Date', 'Outdoor Temperature (°C)', 'Feels Like (°C)', 'Wind Speed (m/sec)', 'Wind Gust (m/sec)', 'Max Daily Gust (m

In [4]:
# --- 2. standardize column names ---
def clean_colnames(df):
    df = df.rename(columns=lambda x: x.strip().lower().replace(" ", "_").replace("(", "").replace(")", "").replace("%","pct"))
    return df

In [5]:
# --- 3. full load and parse dates ---
def load_and_prepare(path):
    df = pd.read_csv(path)
    df = clean_colnames(df)

    # Identifica qual coluna de data existe
    date_col = "simple_date" if "simple_date" in df.columns else "date"

    # Converte pra datetime
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce", utc=True)

    # Renomeia a coluna escolhida para 'date'
    df.rename(columns={date_col: "date"}, inplace=True)

    # Remove a outra, se existir
    for col in ["simple_date", "date"]:
        if col != "date" and col in df.columns:
            df.drop(columns=col, inplace=True)

    # Define o índice e ordena
    df = df.set_index("date").sort_index()

    return df

In [6]:
df1 = load_and_prepare(files["Z1"])
df2 = load_and_prepare(files["Z2"])
df3 = load_and_prepare(files["Z3"])

In [7]:
set1, set2, set3 = set(df1.columns), set(df2.columns), set(df3.columns)

print("Só no Z1 vs Z2:", set1 - set2)
print("Só no Z2 vs Z1:", set2 - set1)
print("Só no Z1 vs Z3:", set1 - set3)
print("Só no Z3 vs Z1:", set3 - set1)
print("Só no Z2 vs Z3:", set2 - set3)
print("Só no Z3 vs Z2:", set3 - set2)

Só no Z1 vs Z2: {'yearly_rain_mm'}
Só no Z2 vs Z1: {'co2_battery'}
Só no Z1 vs Z3: {'yearly_rain_mm', 'event_rain_mm'}
Só no Z3 vs Z1: set()
Só no Z2 vs Z3: {'co2_battery', 'event_rain_mm'}
Só no Z3 vs Z2: set()


In [8]:
common_cols = set1 & set2 & set3
print("Comuns a todos:", common_cols)

Comuns a todos: {'wind_gust_m/sec', 'max_daily_gust_m/sec', 'wind_speed_m/sec', 'weekly_rain_mm', 'rain_rate_mm/hr', 'humidity_pct', 'daily_rain_mm', 'ultra-violet_radiation_index', 'feels_like_°c', 'monthly_rain_mm', 'outdoor_temperature_°c', 'solar_radiation_w/m^2', 'absolute_pressure_mmhg', 'wind_direction_°', 'relative_pressure_mmhg', 'total_rain_mm'}


In [9]:
# --- 4. harmonizar colunas entre os dois (mapear nomes equivalentes)
# Exemplo: padronizar nomes comuns (ajuste conforme suas colunas)
rename_map = {
    "outdoor_temperature_°c": "temp_c",
    "feels_like_°c": "feels_like_c",
    "humidity_%": "humidity_pct",
    "wind_speed_m/sec": "wind_speed_m_s",
    "daily_rain_mm": "daily_rain_mm",
    "absolute_pressure_mmhg": "abs_pressure_mmhg",
    "relative_pressure_mmhg": "rel_pressure_mmhg",
    "solar_radiation_w/m^2": "solar_w_m2",
    "wind_direction_°": "wind_dir_deg",
    "rain_rate_mm/hr": "rain_rate_mm_h",
    "wind_gust_m/sec": "wind_gust_m_s",
    "max_wind_speed_m/sec": "max_wind_speed_m_s",
    "max_daily_gust_m/sec": "max_daily_gust_m_s",
    "simple_date": "date"
}

df1.rename(columns=rename_map, inplace=True)
df2.rename(columns=rename_map, inplace=True)
df3.rename(columns=rename_map, inplace=True)

In [10]:
cols_miss_1 = set(df1.columns) - set(rename_map.values())
cols_miss_2 = set(df2.columns) - set(rename_map.values())
cols_miss_3 = set(df3.columns) - set(rename_map.values())

print("Colunas não mapeadas no Z1:", cols_miss_1)
print("Colunas não mapeadas no Z2:", cols_miss_2)
print("Colunas não mapeadas no Z3:", cols_miss_3)

Colunas não mapeadas no Z1: {'ultra-violet_radiation_index', 'weekly_rain_mm', 'event_rain_mm', 'yearly_rain_mm', 'monthly_rain_mm', 'total_rain_mm'}
Colunas não mapeadas no Z2: {'ultra-violet_radiation_index', 'weekly_rain_mm', 'event_rain_mm', 'co2_battery', 'monthly_rain_mm', 'total_rain_mm'}
Colunas não mapeadas no Z3: {'ultra-violet_radiation_index', 'weekly_rain_mm', 'total_rain_mm', 'monthly_rain_mm'}


In [11]:
all_cols = set(df1.columns) | set(df2.columns) | set(df3.columns)
cols_missing = all_cols - set(rename_map.values())
print("Colunas não mapeadas em nenhum dataset:", cols_missing)

Colunas não mapeadas em nenhum dataset: {'weekly_rain_mm', 'ultra-violet_radiation_index', 'event_rain_mm', 'co2_battery', 'yearly_rain_mm', 'monthly_rain_mm', 'total_rain_mm'}


In [12]:
common_cols = sorted(set(df1.columns) | set(df2.columns) | set(df3.columns))
df1 = df1.reindex(columns=common_cols)
df2 = df2.reindex(columns=common_cols)
df3 = df3.reindex(columns=common_cols)

In [13]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42347 entries, ('2025-06-04T00:00:00-05:00', Timestamp('2025-06-04 00:00:00+0000', tz='UTC')) to ('2025-11-04T23:55:00-05:00', Timestamp('2025-11-04 23:55:00+0000', tz='UTC'))
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   abs_pressure_mmhg             42347 non-null  float64
 1   co2_battery                   0 non-null      float64
 2   daily_rain_mm                 42340 non-null  float64
 3   event_rain_mm                 0 non-null      float64
 4   feels_like_c                  42340 non-null  float64
 5   humidity_pct                  42340 non-null  float64
 6   max_daily_gust_m_s            42340 non-null  float64
 7   monthly_rain_mm               42340 non-null  float64
 8   rain_rate_mm_h                42340 non-null  float64
 9   rel_pressure_mmhg             42347 non-null  float64
 10  solar_w_m2                    423

In [14]:
# insert a 'zone' column (set as first column) so each dataframe carries its zone id
df1.insert(0, "zone", 1)
df2.insert(0, "zone", 2)
df3.insert(0, "zone", 3)

df = pd.concat([df1, df2, df3])

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 130338 entries, ('2025-06-04T00:00:00-05:00', Timestamp('2025-06-04 00:00:00+0000', tz='UTC')) to ('2025-11-04T23:55:00-05:00', Timestamp('2025-11-04 23:55:00+0000', tz='UTC'))
Data columns (total 20 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   zone                          130338 non-null  int64  
 1   abs_pressure_mmhg             130338 non-null  float64
 2   co2_battery                   44030 non-null   float64
 3   daily_rain_mm                 130009 non-null  float64
 4   event_rain_mm                 87991 non-null   float64
 5   feels_like_c                  130009 non-null  float64
 6   humidity_pct                  130009 non-null  float64
 7   max_daily_gust_m_s            130009 non-null  float64
 8   monthly_rain_mm               130009 non-null  float64
 9   rain_rate_mm_h                130009 non-null  float64
 10  rel_pressure_mmhg   

In [16]:
df.drop(columns=['co2_battery', 'yearly_rain_mm'], errors='ignore', inplace=True)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 130338 entries, ('2025-06-04T00:00:00-05:00', Timestamp('2025-06-04 00:00:00+0000', tz='UTC')) to ('2025-11-04T23:55:00-05:00', Timestamp('2025-11-04 23:55:00+0000', tz='UTC'))
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   zone                          130338 non-null  int64  
 1   abs_pressure_mmhg             130338 non-null  float64
 2   daily_rain_mm                 130009 non-null  float64
 3   event_rain_mm                 87991 non-null   float64
 4   feels_like_c                  130009 non-null  float64
 5   humidity_pct                  130009 non-null  float64
 6   max_daily_gust_m_s            130009 non-null  float64
 7   monthly_rain_mm               130009 non-null  float64
 8   rain_rate_mm_h                130009 non-null  float64
 9   rel_pressure_mmhg             130338 non-null  float64
 10  solar_w_m2          

In [18]:
def to_numeric_cols(df):
    for c in df.columns:
        if df[c].dtype == object:
            df[c] = df[c].str.strip().replace({'-':np.nan, '--':np.nan, '':np.nan})
        df[c] = pd.to_numeric(df[c], errors="ignore")
    return df

df = to_numeric_cols(df)

/var/folders/_0/wz7717fn7y18crn9fs47ksyh0000gn/T/ipykernel_91659/1288744439.py:5: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 130338 entries, ('2025-06-04T00:00:00-05:00', Timestamp('2025-06-04 00:00:00+0000', tz='UTC')) to ('2025-11-04T23:55:00-05:00', Timestamp('2025-11-04 23:55:00+0000', tz='UTC'))
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   zone                          130338 non-null  int64  
 1   abs_pressure_mmhg             130338 non-null  float64
 2   daily_rain_mm                 130009 non-null  float64
 3   event_rain_mm                 87991 non-null   float64
 4   feels_like_c                  130009 non-null  float64
 5   humidity_pct                  130009 non-null  float64
 6   max_daily_gust_m_s            130009 non-null  float64
 7   monthly_rain_mm               130009 non-null  float64
 8   rain_rate_mm_h                130009 non-null  float64
 9   rel_pressure_mmhg             130338 non-null  float64
 10  solar_w_m2          

In [20]:
def mark_outliers_iqr(df, col, k=1.5):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    low = q1 - k*iqr
    high = q3 + k*iqr
    return df[(df[col] < low) | (df[col] > high)]

In [21]:
# save including the index (useful if index is zone/date MultiIndex)
df.to_csv("zones_combined_cleaned.csv", index=True)